## Save zarr stores for some variables
Chug certain variables into a zarr store in `/work` for later use.

In [2]:
import xarray as xr
import gfdl_utils as gu
from dask.diagnostics import ProgressBar

In [6]:
config_id = 'ESM4_historical_D1'
outdir = '/work/gam/projects/oxygen/data/'
filename = 'ds.'+config_id+'.zarr'

In [4]:
dmget = False

ds = xr.Dataset()
if (config_id == 'ESM4_piControl_D') or (config_id == 'ESM4_historical_D1'):

    pathDict = {'pp':('/archive/oar.gfdl.cmip6/ESM4/DECK/'+config_id+'/'+
                        'gfdl.ncrc4-intel16-prod-openmp/pp'),
                'out':'ts',
                'local':'annual/5yr',
                'time':'*'}
    pathDict['ppname']='ocean_cobalt_omip_tracers_year_z_1x1deg'
    variables = ['o2','o2sat']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    
    pathDict['ppname']='ocean_annual_z_1x1deg'
    variables = ['thetao','so','volcello']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    gridpath=gu.core.get_pathstatic(pathDict['pp'],pathDict['ppname'])
    if dmget:
        gu.core.issue_dmget(gridpath)
    else:
        grid = xr.open_dataset(gridpath)
    
if dmget==False:
    ds = ds.chunk({'time':1}).squeeze()

In [5]:
with ProgressBar():
    ds.to_zarr(outdir+filename,mode='w')

[########################################] | 100% Completed | 20.3s
